In [1]:
import numpy as np
import CA as ca
from math import pi
from scipy import signal
from matplotlib import pyplot as plt

# References:
# [1] P.P. Vaidyanathan, ROBUST DIGITAL FILTER STRUCTURES, in HANDBOOK FOR DIGITAL SIGNAL PROCESSING. S.K. Mitra and J.F. Kaiser Eds. Wiley-Interscience, N.Y., 1993, Chapter 7.
# [2] P.P. Vaidyanathan, MULTIRATE SYSTEMS AND FILTER BANKS, Prentice Hall, N.Y., Englewood Cliffs, NJ, 1993, Chapter 3.

In [2]:
def numrecursion(r, N, c):
    q = np.zeros(N + 1, dtype=complex)

    # Initialize recursion
    q[0] = np.sqrt(-r[0] / c, dtype=complex)
    q[N] = np.conj(c * q[0], dtype=complex)
    q[1] = -r[1] / (2 * c * q[0])
    q[N - 1] = np.conj(c * q[1], dtype=complex)

    # The limit of the for loop depends on the order being odd or even
    for n in range(3, int(np.ceil(N / 2)) + 1):
        q[n - 1] = (-r[n - 1] / c - np.dot(q[1:n - 1], q[n - 2:0:-1])) / (2 * q[0])
        q[N + 1 - n - 1] = np.conj(c * q[n - 1])

    # Compute middle coefficient separately when order is even
    if N % 2 == 0:
        q[int((N + 2) / 2) - 1] = (-r[int((N + 2) / 2) - 1] / c - np.dot(q[1:int((N + 2) / 2) - 1],
                                                                         q[int((N + 2) / 2) - 2:0:-1])) / (2 * q[0])
    return q


def powercompnum(b, r, N, c=1):
    """
    Compute numerator of power complementary filter.
    """
    # Check if b is real and c has default value
    if np.isreal(b).all() and c == 1:
        # Try to get a real q with c = 1 and c = -1
        q = numrecursion(r, N, 1)

        if not np.isreal(q).all():
            q = numrecursion(r, N, -1)

        if not np.isreal(q).all():
            print('A real power complementary filter could not be found')
            return q
    else:
        if np.size(c) > 1:
            print('C must be a complex scalar.')
            return None
        if abs(c) - 1 > np.power(np.finfo(float).eps, 2/3):
            print('C must be of magnitude one.')
            return None

        q = numrecursion(r, N, c)

    return q

def auxpoly(b, a):
    revb = np.conj(b[::-1])
    reva = np.conj(a[::-1])

    r = np.subtract(np.convolve(revb, b), np.convolve(a, reva))

    return r

def iirpowcomp(b, a):
    # Find the auxiliary polynomial R(z)
    r = auxpoly(b, a)

    # Compute the numerator of the power complementary transfer function
    q = powercompnum(b, r, len(b) - 1)
    return q, a

def sortnums(b, bp):
    # Sort numerators prior to calling ALLPASSDECOMPOSITION.
    # ALLPASSDECOMPOSITION always requires the first argument to
    # be symmetric. The second argument can be symmetric or antisymmetric.

    # If b is real and antisymmetric, make it the second argument
    if np.max(np.abs(np.add(b, b[::-1]))) < np.finfo(float).eps ** (2 / 3):
        p = bp
        q = b
    else:
        p = b
        q = bp

    return p, q

def allpassdecomposition(p, q, a):
    # If q is real and antisymmetric, make it imaginary
    if np.all(np.isreal(q)):
        if np.max(np.abs(np.add(q, q[::-1]))) < np.finfo(float).eps ** (2 / 3):
            q = q * 1j

    z = np.roots(p - 1j * q)

    # Initialize the allpass functions
    d1 = np.array([1.0])
    d2 = np.array([1.0])

    # Separate the zeros inside the unit circle and the ones outside to form the allpass functions
    for n in range(len(z)):
        if np.abs(z[n]) < 1:
            d2 = np.convolve(d2, [1, -z[n]])
        else:
            d1 = np.convolve(d1, [1, -1 / np.conj(z[n])])

    # Remove roundoff imaginary parts
    d1 = imagprune(d1)
    d2 = imagprune(d2)

    beta = np.sum(d2) * (np.sum(p) + 1j * np.sum(q)) / np.sum(a) / np.sum(np.conj(d2))

    return d1, d2, beta

def imagprune(poly):
    # Function to remove roundoff imaginary parts
    real_part = np.real(poly)
    imag_part = np.imag(poly)
    imag_part[np.abs(imag_part) < np.finfo(float).eps] = 0
    pruned_poly = real_part + imag_part * 1j
    return pruned_poly

In [3]:
def tf2ca(b, a):
    bp, a = iirpowcomp(b, a)
    p, q = sortnums(b, bp)
    d1, d2, beta = allpassdecomposition(p,q,a)
    return d1, d2, beta

In [4]:
LP, HP = ca.Butterworth30dB(1200, 48000, True)

b = LP[0]
a = LP[1]

d1, d2, beta = tf2ca(b, a)

In [5]:
print("b   : ", b)
print("a   : ", a)
print("d1  : ", d1)
print("d2  : ", d2)
print("beta: ", beta)

b   :  [2.34099149e-06 1.17049575e-05 2.34099149e-05 2.34099149e-05
 1.17049575e-05 2.34099149e-06]
a   :  [ 1.         -4.49183097  8.09405542 -7.31208128  3.31104756 -0.60111582]
d1  :  [ 1.        +0.00000000e+00j  1.46543445+7.73170665e-07j
  0.47578439+1.24893655e-06j -0.11112487+5.57818739e-07j]
d2  :  [ 1.        +0.00000000e+00j -0.71847053-2.53722609e-06j
  0.11112487-1.13242262e-07j]
beta:  (-0.39724155506711745-103497.35859487607j)
